# Viola Jones

Detekcja wybranych zdjęć ze zbioru danych

In [ ]:
import cv2
def detect_on_img(filename):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'viola/haarcascade_frontalface_default.xml')
    img = cv2.imread(filename)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    cv2.namedWindow('Face Detection')
    cv2.imshow('Face Detection', img)
    if cv2.waitKey(0):
        cv2.destroyAllWindows()

In [ ]:
file1 = 'WIDER_train/images/8--Election_Campain/8_Election_Campain_Election_Campaign_8_15.jpg'
detect_on_img(file1)

In [ ]:
file2 = 'WIDER_train/images/56--Voter/56_Voter_peoplevoting_56_48.jpg'
detect_on_img(file2)

Detekcja na wideo

In [ ]:
def detect_on_video():
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    camera = cv2.VideoCapture(0)
    while (True):
        ret, frame = camera.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        cv2.imshow("camera", frame)
        if cv2.waitKey(1) == 27: # Escape
            break
        success, frame = camera.read()

In [ ]:
detect_on_video()

# SSD

https://github.com/weiliu89/caffe/tree/ssd/

https://github.com/PacktPublishing/Learning-OpenCV-4-Computer-Vision-with-Python-Third-Edition/blob/master/chapter10/faces_dnn.py

From book:

"Learning-OpenCV-4-Computer-Vision-With-Python-3-Get-to-Grips-With-Tools-Techniques-And-Algorithms-for-Computer-Vision-and-Machine-Learning-by-Joseph"

Detekcja wybranych zdjęć ze zbioru danych

In [ ]:
import cv2
import numpy as np

img = cv2.imread('WIDER_train/images/56--Voter/56_Voter_peoplevoting_56_48.jpg')

face_model = cv2.dnn.readNetFromCaffe(
    'ssd/deploy.prototxt',
    'ssd/res10_300x300_ssd_iter_140000.caffemodel')

In [ ]:
def ssd_detect_on_img(ssd_model, image):

    face_blob_height = 300
    face_average_color = (104, 177, 123)
    face_confidence_threshold = 0.9

    while not img is None:
        h, w = img.shape[:2]
        aspect_ratio = w/h

        # Detect faces in the frame.

        face_blob_width = int(face_blob_height * aspect_ratio)
        face_blob_size = (face_blob_width, face_blob_height)

        face_blob = cv2.dnn.blobFromImage(
            img, size=face_blob_size, mean=face_average_color)

        face_model.setInput(face_blob)

        face_results = face_model.forward()

        # Iterate over the detected faces.
        for face in face_results[0, 0]:
            face_confidence = face[2]
            if face_confidence > face_confidence_threshold:

                # Get the face coordinates.
                x0, y0, x1, y1 = (face[3:7] * [w, h, w, h]).astype(int)

                # Draw a blue rectangle around the face.
                cv2.rectangle(img, (x0, y0), (x1, y1),
                                (255, 0, 0), 2)

        cv2.imshow('Faces', img)

        if cv2.waitKey(1) == 27: # Escape
            cv2.destroyAllWindows()
            break

In [ ]:
ssd_detect_on_img(face_model, img)

Detekcja na wideo

In [ ]:
import cv2
import numpy as np

face_model = cv2.dnn.readNetFromCaffe(
    'ssd/deploy.prototxt',
    'ssd/res10_300x300_ssd_iter_140000.caffemodel')

In [ ]:
def ssd_detect_on_video(ssd_model):

    face_blob_height = 300
    face_average_color = (104, 177, 123)
    face_confidence_threshold = 0.9

    cap = cv2.VideoCapture(0)

    success, frame = cap.read()
    while success:

        h, w = frame.shape[:2]
        aspect_ratio = w/h

        # Detect faces in the frame.

        face_blob_width = int(face_blob_height * aspect_ratio)
        face_blob_size = (face_blob_width, face_blob_height)

        face_blob = cv2.dnn.blobFromImage(
            frame, size=face_blob_size, mean=face_average_color)

        face_model.setInput(face_blob)
        print(frame.shape)
        face_results = face_model.forward()
        print(face_results)
        # Iterate over the detected faces.
        for face in face_results[0, 0]:
            face_confidence = face[2]
            if face_confidence > face_confidence_threshold:

                # Get the face coordinates.
                x0, y0, x1, y1 = (face[3:7] * [w, h, w, h]).astype(int)

                # Draw a blue rectangle around the face.
                cv2.rectangle(frame, (x0, y0), (x1, y1),
                            (255, 0, 0), 2)

        cv2.imshow('Faces', frame)
        if cv2.waitKey(1) == 27: # Escape
            cv2.destroyAllWindows()
            break

        success, frame = cap.read()

In [ ]:
ssd_detect_on_video(face_model)

# TODO: R-CNN

Detekcja wybranych zdjęć ze zbioru danych

Detekcja na wideo